# Customer Churn Analysis - RNN

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Get the dataset

In [2]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataset.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


## Part 1 - Data Preprocessing

In [4]:
# grab all our independent variables
features = ['CreditScore', 'Geography','Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary']

In [5]:
fitler = [dataset.columns.get_loc(c) for c in dataset.columns if c in features]

In [6]:
X = dataset.iloc[:, fitler].values  # create the matrix of features 
y = dataset.iloc[:, 13].values

In [7]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:] # dummy variable trap

In [8]:
# Split the data into the Training and Test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [9]:
# Feature Scaling this eases computer calculations and we don't want any independant variables dominating others
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Make the ANN

In [10]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense # required to build the layers of our ANN

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
# Initialising the ANN - add layers afterwards
classifier = Sequential()

In [12]:
# Adding the input layer and the first hidden layer
# We randomly initialise the weights to small non-zero numbers done by the Dense method.
# input dim is the number of features we have 
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11)) 

In [13]:
## Adding the second hidden layer
## don't need to specify the unput dim anymore as per documentation
classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu')) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=6, kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until


In [14]:
## Adding the third hidden layer
## Tested and Accuracy is higher with only 2 layers
# classifier.add(Dense(output_dim = 6, init = 'uniform', activation = 'relu')) 

In [15]:
# Adding the output layer
# we only want 1 output since our result is binary
# use sigmoid activation function for output layer to get probabilities
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [16]:
# Compiling the ANN
# Adam finds the best weights via 'stochastic gradient descent'
# loss should relate to the output layer activation function, catergorical_crossentropy for non binary classification
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [17]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100) # update weights at 10 # can change

Epoch 1/100
8000/8000 [==============================] - 1s 102us/step - loss: 0.4810 - acc: 0.7960
Epoch 2/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4291 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4241 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4200 - acc: 0.8149
Epoch 5/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4173 - acc: 0.8245
Epoch 6/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4149 - acc: 0.8289
Epoch 7/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4130 - acc: 0.8315
Epoch 8/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4124 - acc: 0.8332
Epoch 9/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4113 - acc: 0.8320
Epoch 10/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4105 - acc: 0.8327
Epoch 11

8000/8000 [==============================] - 1s 76us/step - loss: 0.4015 - acc: 0.8344
Epoch 84/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4015 - acc: 0.8357
Epoch 85/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4010 - acc: 0.8359
Epoch 86/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4011 - acc: 0.8350
Epoch 87/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4014 - acc: 0.8349
Epoch 88/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4012 - acc: 0.8344
Epoch 89/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4008 - acc: 0.8347
Epoch 90/100
8000/8000 [==============================] - 1s 75us/step - loss: 0.4012 - acc: 0.8347
Epoch 91/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4011 - acc: 0.8344
Epoch 92/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4011 - acc: 0.8344
Epoch 93/100


In [18]:
# Part 3 - Making the predictions and evaluating the model

# Predicting the Test set results
y_pred_perc = classifier.predict(X_test)
y_pred = (y_pred_perc > 0.5) # to turn probabilities into true or false

In [19]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [20]:
acc = (cm[0][0]+cm[1][1])/cm.sum()
acc
# 1 hiden layer roughly 0.838
# 2 hiden layer roughly 0.8605
# 3 hiden layer roughly 0.8445
# These values change every time the model is evaluated because 
# the neural network is initialised with random values of gains or weights

0.8415

In [21]:
# 0.8595